<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/04_topic_extraction_with_word_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 単語ベクトルを利用したトピック抽出

## 手順
* テキストを埋め込む。
* embeddingsをクラスタリングする。
  * トピック抽出のつもり。
* クラスタの重心に近い順に単語をソートする。
  * 各トピックを表す単語のつもり。

## 準備

* ランタイムのタイプはCPUでよい。

In [ ]:
!python -m spacy download ja_core_news_md

## データセット

### livedoorニュースコーパスのダウンロード

In [ ]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

### 前処理

In [ ]:
import re
import tarfile

tar_fname = "ldcc-20140209.tar.gz"

def read_title(f):
  next(f) # URL
  next(f) # タイムスタンプ
  title = next(f) # 3行目を返す：タイトル
  title = title.decode('utf-8')
  brackets_tail = re.compile('【[^】]*】$')
  brackets_head = re.compile('^【[^】]*】')
  return re.sub(brackets_head, "", re.sub(brackets_tail, "", title))[:-1]

corpus = []
with tarfile.open(tar_fname) as tf:
  for item in tf:
    if "LICENSE.txt" in item.name:
      continue
    if len(item.name.split('/')) < 3:
      continue
    if not item.name.endswith(".txt"):
      continue
    fname = item.name
    # 今回はクラス名は要らない
    #class_name = fname.split('/')[1]
    f = tf.extractfile(fname)
    title = read_title(f)
    corpus.append(title)

In [ ]:
len(corpus)

## テキストの埋め込み

In [ ]:
from tqdm import tqdm
import numpy as np
import spacy

nlp = spacy.load('ja_core_news_md')

X = []
for text in tqdm(corpus):
  tokens = nlp(text)
  X.append(tokens.vector)
X = np.array(X)

In [ ]:
X.shape

## 語彙集合の作成

* 語彙集合を作る目的で、形態素解析を行う。
* 今回は、名詞、固有名詞のみを残す。

In [ ]:
pos_list = ["NOUN", "PROPN"]

vocabulary = list()
for text in tqdm(corpus):
  vocabulary += [token.lemma_ for token in nlp(text) if token.pos_ in pos_list]
vocabulary = set(vocabulary)

In [ ]:
embedded_words = list()
word_embeddings = list()
for word in tqdm(vocabulary):
  wv = nlp(word).vector
  if (wv != 0.0).sum() == 0:
    continue
  embedded_words.append(word)
  word_embeddings.append(wv)

In [ ]:
word_embeddings = np.array(word_embeddings)
embedded_words = np.array(embedded_words)

In [ ]:
word_embeddings.shape

## クラスタリング

* 今回はk-meansでクラスタリングする。

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=0, n_init="auto").fit(X)

In [ ]:
kmeans.cluster_centers_.shape

In [ ]:
labels, counts = np.unique(kmeans.labels_, return_counts=True)
for label, count in sorted(list(zip(labels, counts)), key=lambda x: - x[1]):
  print(label, count)

In [ ]:
corpus = np.array(corpus)
corpus[kmeans.labels_ == 5][:10]

## クラスタのラベル付け

* クラスタの重心に近い単語を20個選ぶ。
  * 各トピックを表す単語のつもり。

* 内積で単語を選ぶ。

In [ ]:
for i in range(kmeans.n_clusters):
  similarities = np.dot(word_embeddings, kmeans.cluster_centers_[i])
  print(' '.join(list(embedded_words[(- similarities).argsort()[:20]])))